In [1]:
!pip install -q langchain_community
!pip install -q pypdf
!pip install -q -U langchain-text-splitters
!pip install -q -U langchain
!pip install -q -U bitsandbytes
!pip install -q -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.5/111.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=nf4_config)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [7]:
from langchain_community.document_loaders import PyPDFLoader

pdf = PyPDFLoader("/content/drive/MyDrive/colab_notebooks/Computer_vision.pdf")
pages = pdf.load_and_split()
pages

[Document(page_content='Computer vision\nComputer vision tasks include methods for acquiring, processing, analyzing and unde rstanding digital\nimages, and extraction of high-dimensional data from the real world in order to produc e numerical or\nsymbolic information, e.g. in the forms of decisions.[1][2][3][4] Understanding in this context means the\ntransformation of visual images (the input to the retina in the human analog) into descriptions of the world\nthat make sense to thought  processes and can elicit appropriate action. This image unde rstanding can be\nseen as the disentangling of symbolic information from image data using models constructed with the aid of\ngeometry, phys ics, statistics, and learning theory.\nThe scientific discipline of computer vision is concerned with the theory behind artificial systems that\nextract information from images. The image data can take many forms, such as video sequences, views\nfrom multiple cameras, multi-dimensional data from a 3D scan

In [9]:
from langchain.prompts import PromptTemplate

template = """
<s>[INST]<<SYS>>Answer the question only from the context below, if the answer is not in the context, reply "i dont know".
<</SYS>>
Context: {context}
Query: {query} [/INST]
"""

prompt = PromptTemplate.from_template(template)
fprompt = prompt.format(context="My give name is venkata ratnam, my friends call me Venkat", query="what is my name ?")

fprompt

'\n<s>[INST]<<SYS>>Answer the question only from the context below, if the answer is not in the context, reply "i dont know".\n<</SYS>>\nContext: My give name is venkata ratnam, my friends call me Venkat\nQuery: what is my name ? [/INST]\n'

In [10]:
inputs = tokenizer(fprompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=150)
ans = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print (ans)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(



 [INST]<<SYS>>Answer the question only from the context below, if the answer is not in the context, reply "i dont know".
<</SYS>>
Context: My give name is venkata ratnam, my friends call me Venkat
Query: what is my name ? [/INST]
Your name is Venkata Ratnam, and your friends call you Venkat.


In [11]:
fprompt = prompt.format(context="My give name is venkata ratnam, my friends call me Venkat", query="what is the size of pacific ocean ?")

print (fprompt)
inputs = tokenizer(fprompt, return_tensors="pt")

generate_ids = model.generate(inputs.input_ids, max_length=150)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]



<s>[INST]<<SYS>>Answer the question only from the context below, if the answer is not in the context, reply "i dont know".
<</SYS>>
Context: My give name is venkata ratnam, my friends call me Venkat
Query: what is the size of pacific ocean ? [/INST]



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'\n [INST]<<SYS>>Answer the question only from the context below, if the answer is not in the context, reply "i dont know".\n<</SYS>>\nContext: My give name is venkata ratnam, my friends call me Venkat\nQuery: what is the size of pacific ocean ? [/INST]\nI\'m just an AI, I don\'t have access to real-time information about the size of the Pacific Ocean. However, according to various sources, the Pacific Ocean is the largest of the Earth\'s five oceans, covering an area of approximately 155.6 million square kilometers (60.1 million square'